In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
from os.path import join
from os import getcwd
from IPython.display import clear_output

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, auc

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD, Adam
from keras.metrics import AUC # Area under the curve, default: ROC
from keras.losses import BinaryCrossentropy
from keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint, TensorBoard
from keras.optimizers.schedules import ExponentialDecay
from keras.initializers import GlorotNormal
import kerastuner as kt

In [4]:
from functions import *

In [5]:
checkpoints_path = getcwd()+'\\checkpoints_A'
tensor_path = getcwd()+'\\tensor_A'

In [6]:
df = pd.read_csv('../../databases/diabetes.csv')
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [7]:
outlayers = {
    'BloodPressure': (40, np.Inf),
    'SkinThickness': (0, 80),
    'Insulin': (0, 400),
    'BMI': (0, 50)
}

zeros = [
    'Glucose',
    'BloodPressure',
    'SkinThickness',
    'Insulin',
    'BMI'
]
x_df = df[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin','BMI', 'DiabetesPedigreeFunction', 'Age']]
y_df = df['Outcome']

In [8]:
# Split dataset into 15% test, 85% train 
x_temp, x_test, y_temp, y_test = train_test_split(x_df, y_df, test_size=0.15, random_state=1)
x_train, x_valid, y_train, y_valid = train_test_split(x_temp, y_temp, test_size=0.15, random_state=1)

# Regresión Logistica

### Con los datos sin procesar

In [9]:
name = 'basic'
checkdir = join(checkpoints_path,name)
tensordir = join(tensor_path, name)

model_0 = Sequential()
model_0.add(Dense(units=1, activation='sigmoid', input_shape=(x_train.shape[1],)))
model_0.compile(optimizer=SGD(), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

In [10]:
model_0_checkpoint_callback = ModelCheckpoint(
    filepath=checkdir,
    save_weights_only=True,
    monitor='val_auc',
    mode='max',
    save_best_only=True)

tensor_call = TensorBoard(log_dir=tensordir)

In [11]:
%%time
history = model_0.fit(x_train, y_train, epochs=50, 
            validation_data=(x_valid, y_valid),
            callbacks=[model_0_checkpoint_callback, tensor_call],
            verbose=0)
# Cargo el mejor modelo entrenado
model_0.load_weights(checkdir)
verify_model(model_0, x_train, y_train, x_valid, y_valid)

Instructions for updating:
use `tf.profiler.experimental.stop` instead.
Wall time: 15.1 s


,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.657407,0.423823,0.808290,0.428571,0.805263
1,Validacion,0.731158,0.468750,0.911765,0.476923,0.909091


### Reemplazando datos invalidos

In [12]:
x_train_nonzero, _data = replace_outliers_zeros(x_train, {}, zeros, mean_median=True)
x_test_nonzero, _data = replace_outliers_zeros(x_test, {}, zeros, mean_median=True, data_to_replace=_data)
x_valid_nonzero, _data = replace_outliers_zeros(x_valid, {}, zeros, mean_median=True, data_to_replace=_data)

In [13]:
name = 'valid'
checkdir = join(checkpoints_path, name)
tensordir = join(tensor_path, name)
tensor_call = TensorBoard(log_dir=tensordir)

model_1 = Sequential()
model_1.add(Dense(units=1, activation='sigmoid', input_shape=(x_train.shape[1],)))
model_1.compile(optimizer=SGD(), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

model_1_checkpoint_callback = ModelCheckpoint(filepath=checkdir,
                                              save_weights_only=True,
                                              monitor='val_auc',
                                              mode='max',
                                              save_best_only=True)

In [14]:
%%time 
history = model_1.fit(x_train_nonzero, y_train, epochs=50, validation_data=(x_valid_nonzero, y_valid), callbacks=[model_1_checkpoint_callback, tensor_call], verbose=0)
# Cargo el mejor modelo entrenado
model_1.load_weights(checkdir)
verify_model(model_1, x_train_nonzero, y_train, x_valid_nonzero, y_valid)

Wall time: 14.8 s


,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.676891,0.808864,0.46114,0.563291,0.737374
1,Validacion,0.709329,0.906250,0.50000,0.739130,0.773333


Se puede observar como en ambos casos se obtuvo un mejor resultado en validacion que en train, lo cual es un comportamiento que se puede considerar poco esperado

### Reemplazando outlayers

In [15]:
x_train_clean, _data = replace_outliers_zeros(x_train, outlayers, zeros, mean_median=True)
x_test_clean, _data = replace_outliers_zeros(x_test, outlayers, zeros, mean_median=True, data_to_replace=_data)
x_valid_clean, _data = replace_outliers_zeros(x_valid, outlayers, zeros, mean_median=True, data_to_replace=_data)

In [16]:
name = 'sin_outlayers'
checkdir = join(checkpoints_path, name)
tensordir = join(tensor_path, name)
tensor_call = TensorBoard(log_dir=tensordir)

model_2 = Sequential()
model_2.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_clean.shape[1],)))
model_2.compile(optimizer=SGD(), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

model_2_checkpoint_callback = ModelCheckpoint(filepath=checkdir,
                                              save_weights_only=True,
                                              monitor='val_auc',
                                              mode='max',
                                              save_best_only=True)

In [17]:
%%time 
history = model_2.fit(x_train_clean, y_train, epochs=50, validation_data=(x_valid_clean, y_valid), callbacks=[model_2_checkpoint_callback, tensor_call], verbose=0)
# Cargo el mejor modelo entrenado
model_2.load_weights(checkdir)
verify_model(model_2, x_train_clean, y_train, x_valid_clean, y_valid)

Wall time: 14.4 s


,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.670461,0.828255,0.430052,0.572414,0.731051
1,Validacion,0.738051,0.921875,0.411765,0.736842,0.746835


### Normalizando los datos de entrada

In [18]:
x_train_norm, _norm_dict = normalize(x_train_clean, None)
x_valid_norm, _norm_dict = normalize(x_valid_clean, _norm_dict)
x_test_norm, _norm_dict = normalize(x_test_clean, _norm_dict)

In [19]:
name = 'normalizado'
checkdir = join(checkpoints_path, name)
tensordir = join(tensor_path, name)
tensor_call = TensorBoard(log_dir=tensordir)

model_3 = Sequential()
model_3.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_norm.shape[1],)))
model_3.compile(optimizer=SGD(), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

model_3_checkpoint_callback = ModelCheckpoint(filepath=checkdir,
                                              save_weights_only=True,
                                              monitor='val_auc',
                                              mode='max',
                                              save_best_only=True)

In [20]:
%%time 
history = model_3.fit(x_train_norm, y_train, epochs=100, validation_data=(x_valid_norm, y_valid), callbacks=[model_3_checkpoint_callback, tensor_call], verbose=1)
# Cargo el mejor modelo entrenado
model_3.load_weights(checkdir)
verify_model(model_3, x_train_norm, y_train, x_valid_norm, y_valid)

Epoch 1/100
18/18 [==============================] - 2s 120ms/step - loss: 0.8324 - auc: 0.4407 - val_loss: 0.7719 - val_auc: 0.5016
Epoch 2/100
18/18 [==============================] - 1s 29ms/step - loss: 0.7882 - auc: 0.4815 - val_loss: 0.7341 - val_auc: 0.5340
Epoch 3/100
18/18 [==============================] - 1s 31ms/step - loss: 0.7487 - auc: 0.5236 - val_loss: 0.7019 - val_auc: 0.5744
Epoch 4/100
18/18 [==============================] - 1s 31ms/step - loss: 0.7152 - auc: 0.5621 - val_loss: 0.6734 - val_auc: 0.6101
Epoch 5/100
18/18 [==============================] - 1s 40ms/step - loss: 0.6856 - auc: 0.6025 - val_loss: 0.6501 - val_auc: 0.6438
Epoch 6/100
18/18 [==============================] - 1s 31ms/step - loss: 0.6610 - auc: 0.6356 - val_loss: 0.6301 - val_auc: 0.6719
Epoch 7/100
18/18 [==============================] - 1s 31ms/step - loss: 0.6402 - auc: 0.6676 - val_loss: 0.6130 - val_auc: 0.7020
Epoch 8/100
18/18 [==============================] - 1s 29ms/step - loss: 0

,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.836981,0.869806,0.580311,0.704403,0.794937
1,Validacion,0.826287,0.890625,0.529412,0.720000,0.780822


### Early Stopping
Los hyperparámetrso principales de este callback son:
- monitor: indica la variable a monitorear
- min_delta: la minima diferencia que se considera como mejora
- patience: cantidad de epochs sin mejoras antes de parar

In [21]:
stop_early = EarlyStopping(monitor='val_auc', patience=20)
name = 'early_stop'
checkdir = join(checkpoints_path, name)
tensordir = join(tensor_path, name)
tensor_call = TensorBoard(log_dir=tensordir)
model_4_checkpoint_callback = ModelCheckpoint(filepath=checkdir,
                                              save_weights_only=True,
                                              monitor='val_auc',
                                              mode='max',
                                              save_best_only=True)

In [22]:
%%time 
history = model_3.fit(x_train_norm, y_train, epochs=50, validation_data=(x_valid_norm, y_valid), callbacks=[model_4_checkpoint_callback, stop_early, tensor_call], verbose=1)
# Cargo el mejor modelo entrenado
model_3.load_weights(checkdir)
verify_model(model_3, x_train_norm, y_train, x_valid_norm, y_valid)

Epoch 1/50
18/18 [==============================] - 2s 91ms/step - loss: 0.4752 - auc: 0.8366 - val_loss: 0.4783 - val_auc: 0.8265
Epoch 2/50
18/18 [==============================] - 0s 8ms/step - loss: 0.4751 - auc: 0.8368 - val_loss: 0.4783 - val_auc: 0.8249
Epoch 3/50
18/18 [==============================] - 0s 9ms/step - loss: 0.4750 - auc: 0.8368 - val_loss: 0.4780 - val_auc: 0.8256
Epoch 4/50
18/18 [==============================] - 0s 10ms/step - loss: 0.4749 - auc: 0.8368 - val_loss: 0.4780 - val_auc: 0.8256
Epoch 5/50
18/18 [==============================] - 0s 8ms/step - loss: 0.4749 - auc: 0.8370 - val_loss: 0.4780 - val_auc: 0.8263
Epoch 6/50
18/18 [==============================] - 0s 13ms/step - loss: 0.4748 - auc: 0.8371 - val_loss: 0.4779 - val_auc: 0.8265
Epoch 7/50
18/18 [==============================] - 0s 12ms/step - loss: 0.4748 - auc: 0.8372 - val_loss: 0.4779 - val_auc: 0.8258
Epoch 8/50
18/18 [==============================] - 0s 9ms/step - loss: 0.4748 - auc: 

,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.836953,0.872576,0.575130,0.707006,0.793451
1,Validacion,0.826287,0.890625,0.529412,0.720000,0.780822


Se puede evidenciar como al utilizar early stopping el entrenamiento finaliza antes debido a que las mejoras son reducidas

## Ajuste de Hyperparámetros

### Learning rate
Por defecto en SGD el learning rate que utiliza keras es 0.01

In [23]:
learning_rates = [1e-5, 1e-4, 1e-3, 0.01, 0.05, 0.1, 0.22, 0.33, 0.47, 0.56, 0.81, 1.2, 1.8, 2.2, 2.7, 3.3, 4.7]

arr_of_metrics = []
auc_res = {'L_r':['AUC', 'Epochs']}
best = 0
best_name = ''
for i in learning_rates:
    name = 'learning_{}'.format(i)
    checkdir = join(checkpoints_path, name)
    tensordir = join(tensor_path, name)
    tensor_call = TensorBoard(log_dir=tensordir)
    learning_checkpoint_callback = ModelCheckpoint(filepath=checkdir,
                                                  save_weights_only=True,
                                                  monitor='val_auc',
                                                  mode='max',
                                                  save_best_only=True)


    learning = Sequential()
    learning.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_norm.shape[1],)))
    learning.compile(optimizer=SGD(learning_rate=i), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

    history = learning.fit(x_train_norm, y_train, epochs=100, validation_data=(x_valid_norm, y_valid), callbacks=[learning_checkpoint_callback, stop_early, tensor_call], 
                          verbose=1)
    # Cargo el mejor modelo entrenado
    learning.load_weights(checkdir)
    metrics = verify_model(learning, x_train_norm, y_train, x_valid_norm, y_valid)
    arr_of_metrics.append(metrics)
    auc_res[i] = [metrics['AUC ROC'][1], history.epoch[-1]]
    if best < auc_res[i][0]:
        best_name = checkdir
        best = auc_res[i][0]
    clear_output(wait=True)
pd.DataFrame(data=auc_res)

,L_r,1e-05,0.0001,0.001,0.01,0.05,0.1,0.22,0.33,0.47,0.56,0.81,1.2,1.8,2.2,2.7,3.3,4.7
0,AUC,0.730699,0.502757,0.438879,0.744945,0.83318,0.838235,0.834559,0.834559,0.840074,0.839614,0.84375,0.846048,0.841452,0.841452,0.837316,0.842371,0.841912
1,Epochs,20.000000,20.000000,20.000000,20.000000,20.00000,20.000000,20.000000,20.000000,20.000000,26.000000,45.00000,33.000000,21.000000,32.000000,21.000000,23.000000,31.000000


Es interesante observar como para los learning rate mas chicos, el entrenamiento se corta debido al early stoping en los primeros 15 epochs, ya que dentro de esos primeros 15 las mejoras son pocas debido a su avance "suave", en comparación con los learning rates mas altos que obtienen una mejor optimización en mayor cantidad de epochs

In [24]:
# Cargo el mejor modelo entrenado
learning.load_weights(best_name)
verify_model(learning, x_train_norm, y_train, x_valid_norm, y_valid)

,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.833867,0.850416,0.637306,0.694915,0.814324
1,Validacion,0.846048,0.875000,0.588235,0.714286,0.800000


#### Scheduling learning rate

In [25]:
learning_schedule = ExponentialDecay(initial_learning_rate=2.7, decay_steps=100, decay_rate=0.96)
name = 'exponetial'
checkdir = join(checkpoints_path, name)
tensordir = join(tensor_path, name)
tensor_call = TensorBoard(log_dir=tensordir)
    
exponential_checkpoint_callback = ModelCheckpoint(filepath=checkdir,
                                                  save_weights_only=True,
                                                  monitor='val_auc',
                                                  mode='max',
                                                  save_best_only=True)


In [26]:
learning = Sequential()
learning.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_norm.shape[1],)))
learning.compile(optimizer=SGD(learning_rate=learning_schedule), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

In [27]:
history = learning.fit(x_train_norm, y_train, epochs=100, validation_data=(x_valid_norm, y_valid), callbacks=[exponential_checkpoint_callback, stop_early, tensor_call], 
                      verbose=1)
# Cargo el mejor modelo entrenado
learning.load_weights(checkdir)
verify_model(learning, x_train_norm, y_train, x_valid_norm, y_valid)

Epoch 1/100
18/18 [==============================] - 3s 142ms/step - loss: 0.5396 - auc: 0.7791 - val_loss: 0.4920 - val_auc: 0.8265
Epoch 2/100
18/18 [==============================] - 0s 20ms/step - loss: 0.5362 - auc: 0.7999 - val_loss: 0.6598 - val_auc: 0.7254
Epoch 3/100
18/18 [==============================] - 0s 14ms/step - loss: 0.5421 - auc: 0.7867 - val_loss: 0.5255 - val_auc: 0.8100
Epoch 4/100
18/18 [==============================] - 1s 37ms/step - loss: 0.5310 - auc: 0.8029 - val_loss: 0.4895 - val_auc: 0.8318
Epoch 5/100
18/18 [==============================] - 0s 11ms/step - loss: 0.5347 - auc: 0.8009 - val_loss: 0.4868 - val_auc: 0.8309
Epoch 6/100
18/18 [==============================] - 0s 11ms/step - loss: 0.5627 - auc: 0.7871 - val_loss: 0.4834 - val_auc: 0.8270
Epoch 7/100
18/18 [==============================] - 1s 33ms/step - loss: 0.5292 - auc: 0.8074 - val_loss: 0.4951 - val_auc: 0.8350
Epoch 8/100
18/18 [==============================] - 0s 10ms/step - loss: 0

,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.810443,0.833795,0.601036,0.659091,0.796296
1,Validacion,0.842371,0.859375,0.617647,0.700000,0.808824


### Momentum

In [28]:
momentum_arr = [0.5, 0.6, 0.85, 0.9, 0.95, 0.99, 0.999]

arr_of_metrics = []
auc_res = {'momentum':['AUC', 'Epochs']}
for i in momentum_arr:
    name = 'momentum_{}'.format(i)
    checkdir = join(checkpoints_path, name)
    tensordir = join(tensor_path, name)
    tensor_call = TensorBoard(log_dir=tensordir)

    momentum_checkpoint_callback = ModelCheckpoint(filepath=checkdir,
                                                  save_weights_only=True,
                                                  monitor='val_auc',
                                                  mode='max',
                                                  save_best_only=True)
    
    mom_model = Sequential()
    mom_model.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_norm.shape[1],)))
    mom_model.compile(optimizer=SGD(learning_rate=learning_schedule, momentum=i), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

    history = mom_model.fit(x_train_norm, y_train, epochs=100, validation_data=(x_valid_norm, y_valid), callbacks=[momentum_checkpoint_callback, tensor_call], 
                          verbose=1)
    # Cargo el mejor modelo entrenado
    mom_model.load_weights(checkdir)
    metrics = verify_model(mom_model, x_train_norm, y_train, x_valid_norm, y_valid)
    arr_of_metrics.append(metrics)
    auc_res[i] = [metrics['AUC ROC'][1], history.epoch[-1]]
    clear_output(wait=True)
pd.DataFrame(data=auc_res)

,momentum,0.5,0.6,0.85,0.9,0.95,0.99,0.999
0,AUC,0.842831,0.840533,0.838235,0.839614,0.829504,0.817555,0.800781
1,Epochs,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000


In [29]:
name = 'momentum'
checkdir = join(checkpoints_path, name)
tensordir = join(tensor_path, name)
tensor_call = TensorBoard(log_dir=tensordir)
momentum_checkpoint_callback = ModelCheckpoint(filepath=checkdir,
                                                  save_weights_only=True,
                                                  monitor='val_auc',
                                                  mode='max',
                                                  save_best_only=True)

mom_model = Sequential()
mom_model.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_norm.shape[1],)))
mom_model.compile(optimizer=SGD(learning_rate=learning_schedule, momentum=0.6), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])
history = mom_model.fit(x_train_norm, y_train, epochs=100, validation_data=(x_valid_norm, y_valid), callbacks=[momentum_checkpoint_callback, stop_early, tensor_call], 
                          verbose=1)
# Cargo el mejor modelo entrenado
mom_model.load_weights(checkdir)
verify_model(mom_model, x_train_norm, y_train, x_valid_norm, y_valid)

Epoch 1/100
18/18 [==============================] - 2s 129ms/step - loss: 0.6320 - auc: 0.7567 - val_loss: 0.6059 - val_auc: 0.7714
Epoch 2/100
18/18 [==============================] - 1s 37ms/step - loss: 0.5912 - auc: 0.7775 - val_loss: 0.5682 - val_auc: 0.7875
Epoch 3/100
18/18 [==============================] - 0s 12ms/step - loss: 0.6135 - auc: 0.7763 - val_loss: 0.8558 - val_auc: 0.5126
Epoch 4/100
18/18 [==============================] - 0s 12ms/step - loss: 0.6385 - auc: 0.7611 - val_loss: 0.5703 - val_auc: 0.7700
Epoch 5/100
18/18 [==============================] - 0s 15ms/step - loss: 0.5991 - auc: 0.7657 - val_loss: 0.5962 - val_auc: 0.7679
Epoch 6/100
18/18 [==============================] - 1s 32ms/step - loss: 0.5681 - auc: 0.7848 - val_loss: 0.5537 - val_auc: 0.8316
Epoch 7/100
18/18 [==============================] - 0s 13ms/step - loss: 0.5698 - auc: 0.7975 - val_loss: 0.7106 - val_auc: 0.6213
Epoch 8/100
18/18 [==============================] - 0s 14ms/step - loss: 0

,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.801731,0.903047,0.487047,0.728682,0.767059
1,Validacion,0.844210,0.875000,0.411765,0.636364,0.736842


### Optimizador Adam

In [30]:
%%time
adam_lr = [1e-5, 1e-4, 1e-3, 0.01, 0.05, 0.1, 0.22, 0.33, 0.47, 0.56, 0.81, 1.2, 1.8, 2.2, 2.7, 3.3, 4.7]

arr_of_metrics = []
auc_res = {'L_r':['AUC', 'Epochs']}
for i in adam_lr:
    name = 'Adam_{}'.format(i)
    checkdir = join(checkpoints_path, name)
    tensordir = join(tensor_path, name)
    tensor_call = TensorBoard(log_dir=tensordir)
    adam_checkpoint_callback = ModelCheckpoint(filepath=checkdir,
                                                  save_weights_only=True,
                                                  monitor='val_auc',
                                                  mode='max',
                                                  save_best_only=True)
    adam_mod = Sequential()
    adam_mod.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_norm.shape[1],)))
    adam_mod.compile(optimizer=Adam(learning_rate=i), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

    history = adam_mod.fit(x_train_norm, y_train, epochs=100, validation_data=(x_valid_norm, y_valid), callbacks=[adam_checkpoint_callback, tensor_call], 
                          verbose=1, use_multiprocessing=True)
    # Cargo el mejor modelo entrenado
    adam_mod.load_weights(checkdir)
    metrics = verify_model(adam_mod, x_train_norm, y_train, x_valid_norm, y_valid)
    arr_of_metrics.append(metrics)
    auc_res[i] = [metrics['AUC ROC'][1], history.epoch[-1]]
    clear_output(wait=True)
pd.DataFrame(data=auc_res)

Wall time: 7min 35s


,L_r,1e-05,0.0001,0.001,0.01,0.05,0.1,0.22,0.33,0.47,0.56,0.81,1.2,1.8,2.2,2.7,3.3,4.7
0,AUC,0.502298,0.747243,0.803309,0.832721,0.837776,0.841912,0.845129,0.84421,0.844669,0.848805,0.841912,0.844669,0.839614,0.849265,0.841912,0.841452,0.842371
1,Epochs,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.00000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000


In [31]:
%%time
name = 'Adam'
checkdir = join(checkpoints_path, name)
tensordir = join(tensor_path, name)
tensor_call = TensorBoard(log_dir=tensordir)
adam_checkpoint_callback = ModelCheckpoint(filepath=checkdir,
                                                  save_weights_only=True,
                                                  monitor='val_auc',
                                                  mode='max',
                                                  save_best_only=True)

adam_model = Sequential()
adam_model.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_norm.shape[1],)))
adam_model.compile(optimizer=Adam(learning_rate=4.7), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])
history = adam_model.fit(x_train_norm, y_train, epochs=100, validation_data=(x_valid_norm, y_valid), callbacks=[adam_checkpoint_callback, tensor_call], 
                          verbose=1)
# Cargo el mejor modelo entrenado
adam_model.load_weights(checkdir)
verify_model(adam_model, x_train_norm, y_train, x_valid_norm, y_valid)

Epoch 1/100
18/18 [==============================] - 3s 148ms/step - loss: 2.4802 - auc: 0.6826 - val_loss: 2.2955 - val_auc: 0.5756
Epoch 2/100
18/18 [==============================] - 0s 27ms/step - loss: 2.8413 - auc: 0.6485 - val_loss: 1.3418 - val_auc: 0.7999
Epoch 3/100
18/18 [==============================] - 0s 12ms/step - loss: 1.6175 - auc: 0.7226 - val_loss: 1.2660 - val_auc: 0.6939
Epoch 4/100
18/18 [==============================] - 0s 12ms/step - loss: 1.1418 - auc: 0.7231 - val_loss: 1.3970 - val_auc: 0.6866
Epoch 5/100
18/18 [==============================] - 1s 30ms/step - loss: 1.5489 - auc: 0.6725 - val_loss: 0.8779 - val_auc: 0.8293
Epoch 6/100
18/18 [==============================] - 0s 17ms/step - loss: 1.4270 - auc: 0.7221 - val_loss: 1.9991 - val_auc: 0.6108
Epoch 7/100
18/18 [==============================] - 0s 10ms/step - loss: 1.8665 - auc: 0.7040 - val_loss: 1.8621 - val_auc: 0.6073
Epoch 8/100
18/18 [==============================] - 0s 9ms/step - loss: 1.

,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.790206,0.767313,0.668394,0.605634,0.812317
1,Validacion,0.831342,0.796875,0.617647,0.617647,0.796875


Evaluación contra test

In [32]:
print('Utilizando SGD')
verify_model(mom_model, x_train_norm, y_train, x_test_norm, y_test, valid_label='Test')

Utilizando SGD


,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.801731,0.903047,0.487047,0.728682,0.767059
1,Test,0.832520,0.906667,0.487805,0.740741,0.764045


In [33]:
print('Utilizando Adam')
verify_model(adam_model, x_train_norm, y_train, x_test_norm, y_test, valid_label='Test')

Utilizando Adam


,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.790206,0.767313,0.668394,0.605634,0.812317
1,Test,0.827317,0.760000,0.634146,0.590909,0.791667


Finalmente se obtiene que el modelo que utiliza como optimizador SGD y los hyperparámetros optimizados fue el mejor de los entrenados hasta ahora

Cabe destacar una particularidad, que los resustados en Test son mejores que en el set de validacion, e incluso mejores que el set de train, con lo cual concluir que la metrica de nuestro modelo es AUC=0.86 no sería correcto, ya que es el caso en el que tenemos un estimador con alta varianza, por lo tanto una estimación utilizando k-folding sería mas adecuada (si hay tiempo la hago...)

In [34]:
f2_threshold(mom_model, x_train_norm, y_train, x_valid_norm, y_valid)

TypeError: fbeta_score() missing 1 required keyword-only argument: 'beta'